In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# データの読み込み
data_path = '/Users/hayakawakazue/Downloads/data/test/test_cleaned3/test_33_cleaned_最寄駅_面積（㎡）.csv'
data_cleaned = pd.read_csv(data_path)

print(data_cleaned.head())

         ID  市区町村コード  最寄駅：距離（分）   間取り  面積（㎡）     建築年 建物の構造    取引時点
0  33000189    33101       14.0  ２ＬＤＫ   65.0  2007.0    ＲＣ  202006
1  33000471    33101        1.0  ３ＬＤＫ   75.0  2018.0    ＲＣ  202003
2  33000579    33101       23.0   １ＤＫ   70.0  1998.0   ＳＲＣ  202003
3  33000984    33101       13.0  ３ＬＤＫ   80.0  1995.0    ＲＣ  202003
4  33001008    33101       18.0  ４ＬＤＫ   70.0  1980.0   ＳＲＣ  202003


1. 新しい特徴量エンジニアリングを作成する

In [2]:
import pandas as pd
import numpy as np

# データの読み込み
data_path = '/Users/hayakawakazue/Downloads/data/test/test_cleaned3/test_33_cleaned_最寄駅_面積（㎡）.csv'
test_data_cleaned = pd.read_csv(data_path)

# ダミーの取引価格（総額）_log列を追加
test_data_cleaned['取引価格（総額）_log'] = 0  # ダミー値として0を使用

# 築年数を計算
test_data_cleaned['築年数'] = 2024 - test_data_cleaned['建築年']

# 面積/築年数比を計算
test_data_cleaned['面積/築年数比'] = test_data_cleaned['面積（㎡）'] / test_data_cleaned['築年数']

# 価格/面積比を計算
test_data_cleaned['価格/面積比'] = test_data_cleaned['取引価格（総額）_log'] / test_data_cleaned['面積（㎡）']

# 取引時点から年を抽出
test_data_cleaned['取引年'] = test_data_cleaned['取引時点'].astype(str).str[:4].astype(int)

# 新しい特徴量の作成
test_data_cleaned['築年数の二乗'] = test_data_cleaned['築年数'] ** 2
test_data_cleaned['面積×築年数'] = test_data_cleaned['面積（㎡）'] * test_data_cleaned['築年数']

# 取引時点を年と月に分割
test_data_cleaned['取引年'] = test_data_cleaned['取引時点'].apply(lambda x: int(str(x)[:4]))
test_data_cleaned['取引月'] = test_data_cleaned['取引時点'].apply(lambda x: int(str(x)[4:6]))

# 駅距離の統計情報を確認
print(test_data_cleaned['最寄駅：距離（分）'].describe())

# 駅距離を次のように分割して数値に対応付け
bins = [0, 5, 10, 15, 20, 25, 30, np.inf]
numeric_labels = [1, 2, 3, 4, 5, 6, 7]
test_data_cleaned['駅距離_分範囲'] = pd.cut(test_data_cleaned['最寄駅：距離（分）'], bins=bins, labels=numeric_labels)

# エリア人気度スコアの計算
area_popularity = test_data_cleaned.groupby('市区町村コード')['取引価格（総額）_log'].mean().reset_index()
area_popularity.columns = ['市区町村コード', 'エリア人気度スコア']
test_data_cleaned = test_data_cleaned.merge(area_popularity, on='市区町村コード', how='left')

# 年別価格トレンドの計算
yearly_trend = test_data_cleaned.groupby('取引年')['取引価格（総額）_log'].mean().reset_index()
yearly_trend.columns = ['取引年', '年別価格トレンド']
test_data_cleaned = test_data_cleaned.merge(yearly_trend, on='取引年', how='left')

# 価格増加率の計算
test_data_cleaned = test_data_cleaned.sort_values(by='取引時点')
test_data_cleaned['価格増加率'] = test_data_cleaned['取引価格（総額）_log'].pct_change()
test_data_cleaned['価格増加率'].fillna(0, inplace=True)

# 新しい特徴量を含むデータの確認
print(test_data_cleaned.head())

# データを保存
test_data_cleaned_path = '/Users/hayakawakazue/Downloads/data/test/test_cleaned4/test_33_cleaned_feature_engineered.csv'
test_data_cleaned.to_csv(test_data_cleaned_path, index=False)

print(f"データを保存しました: {test_data_cleaned_path}")

# データの行数と列数をプリントする
print(f"\nデータの行数: {test_data_cleaned.shape[0]}")
print(f"データの列数: {test_data_cleaned.shape[1]}")


count    66.000000
mean     15.469697
std      11.109552
min       1.000000
25%       7.000000
50%      14.000000
75%      19.000000
max      45.000000
Name: 最寄駅：距離（分）, dtype: float64
         ID  市区町村コード  最寄駅：距離（分）   間取り  面積（㎡）     建築年 建物の構造    取引時点  \
8  33001344    33101       20.0  ２ＬＤＫ   55.0  1990.0   ＳＲＣ  202003   
1  33000471    33101        1.0  ３ＬＤＫ   75.0  2018.0    ＲＣ  202003   
2  33000579    33101       23.0   １ＤＫ   70.0  1998.0   ＳＲＣ  202003   
3  33000984    33101       13.0  ３ＬＤＫ   80.0  1995.0    ＲＣ  202003   
4  33001008    33101       18.0  ４ＬＤＫ   70.0  1980.0   ＳＲＣ  202003   

   取引価格（総額）_log   築年数    面積/築年数比  価格/面積比   取引年  築年数の二乗  面積×築年数  取引月 駅距離_分範囲  \
8             0  34.0   1.617647     0.0  2020  1156.0  1870.0    3       4   
1             0   6.0  12.500000     0.0  2020    36.0   450.0    3       1   
2             0  26.0   2.692308     0.0  2020   676.0  1820.0    3       5   
3             0  29.0   2.758621     0.0  2020   841.0  2320.0    3       3   
